In [3]:
#setting up the basic environment
import gym 

env = gym.make("Taxi-v3").env

env.render()


+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [4]:
#printing out the action and state space, also resetting the environment
env.reset() 
env.render()

print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space Discrete(6)
State Space Discrete(500)


In [5]:
#printing out a specific state that is related to the inputs of state
state = env.encode(3, 1, 2, 0) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

env.s = state
env.render()

State: 328
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
#printing out the rewards table for state 328
env.P[328]

{0: [(1.0, 428, -1, False)],
 1: [(1.0, 228, -1, False)],
 2: [(1.0, 348, -1, False)],
 3: [(1.0, 328, -1, False)],
 4: [(1.0, 328, -10, False)],
 5: [(1.0, 328, -10, False)]}

In [7]:
#runs a loop that goes until a passenger reaches a desination, represented by having a reward balance of 20
#prints out the steps and the penalties recieved

env.s = 328 

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1
    
    
print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 1923
Penalties incurred: 654


In [8]:
#animates the agent's attempts to transport the passenger
#prints out steps, state, action and reward
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)
        
print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 1923
State: 0
Action: 5
Reward: 20


In [10]:
#initalizing the Q-Table 
import numpy as np
q_table = np.zeros([env.observation_space.n, env.action_space.n])

In [11]:
%%time
"""Training the agent"""

import random
from IPython.display import clear_output

# Hyperparameters            Represents:
alpha = 0.1                 #learning rate
gamma = 0.6                 #Discount Factor
epsilon = 0.1               #Exploration Ratio

# For plotting metrics
all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        if random.uniform(0, 1) < epsilon:           # Tells the agent that it should value the unknown
            action = env.action_space.sample()     
        else:                                        # Otherwise the agent should exploit the known data
            action = np.argmax(q_table[state]) 
                                                     #The following section of code is applying the Q-Learning Algorithm
        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]               #Get current state 
        next_max = np.max(q_table[next_state])           #Get next max reward state
        
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)  #Calculates the most desirable action
        q_table[state, action] = new_value               #Updates the table

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1
        
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

print("Training finished.\n")

Episode: 100000
Training finished.

Wall time: 34 s


In [12]:
#Printing out element of the Q-Table
q_table[328]

array([ -2.40308189,  -2.27325184,  -2.39737445,  -2.35867093,
       -10.95615719, -11.0633733 ])

In [13]:
#This code is simply used to evaluate the agent's performance

total_epochs, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1

    total_penalties += penalties
    total_epochs += epochs

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_epochs / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 13.26
Average penalties per episode: 0.0
